# Calcul de la variable d'intêret : Indicateur synthétique de la production 

( relatif à la partie ***3.2.3 - Calcul de la variable expliquée*** du rapport) 

In [1]:
# Importation des packages 
import os 
import pandas as pd
import numpy as np

In [2]:
# Bases :
path_ID = r'C:\Users\louis\OneDrive\Documents\ENSAE\2A\Stat_app\SAE_2019\Bases_statistiques\Bases_CSV\ID_2019r.csv'
ID=pd.read_csv(path_ID ,sep=';', encoding='latin-1')
path_activ2019 =r'C:\Users\louis\OneDrive\Documents\ENSAE\2A\Stat_app\activ2019_ensae.csv'
activ2019=pd.read_csv(path_activ2019 ,sep=';')
path_ghs_priv= r'C:\Users\louis\OneDrive\Documents\ENSAE\2A\Stat_app\GHS\ghs_pri.csv'
ghs_priv=pd.read_csv(path_ghs_priv ,sep=';', encoding='latin-1')
path_ghs_pub= r'C:\Users\louis\OneDrive\Documents\ENSAE\2A\Stat_app\GHS\ghs_pub.csv'
ghs_pub=pd.read_csv(path_ghs_pub,sep=';', encoding='latin-1')

## Premières analyses de la base activ2019

In [3]:
activ2019

,finess,ghm2,_freq
0,010006526,90Z00Z,223
1,010007300,02C05J,813
2,010007300,02C08J,87
3,010007300,06K02Z,169
4,010007300,06K04J,663
...,...,...,...
153637,980500003,28Z15Z,698
153638,980500003,28Z17Z,552
153639,980500763,28Z04Z,8024
153640,980500763,90Z00Z,1085


Le code ghm "90Z00Z" représente les “Erreurs et autres séjours inclassables”, on ne prendra pas en compte ces ghm pour cette étude. Nous supprimons ces lignes de notre Dataframe. 

In [4]:
Index_90Z00Z= activ2019[ activ2019['ghm2'] == '90Z00Z' ].index
activ2019.drop(Index_90Z00Z , inplace=True)

In [5]:
activ2019

,finess,ghm2,_freq
1,010007300,02C05J,813
2,010007300,02C08J,87
3,010007300,06K02Z,169
4,010007300,06K04J,663
5,010007300,06K05J,63
...,...,...,...
153635,980500003,28Z07Z,1453
153636,980500003,28Z14Z,427
153637,980500003,28Z15Z,698
153638,980500003,28Z17Z,552


In [6]:
activ2019['finess'].nunique()

1416

La base activ2019 est dotée de 152 743 lignes mais ne représente que 1 416 établissements de santé. En effet, un établissement de santé est représenté par un ou plusieurs ghm. 

In [7]:
activ2019.groupby(by=['finess']).size().describe()

count    1416.000000
mean      107.869350
std       169.539057
min         1.000000
25%         4.000000
50%        53.000000
75%       141.000000
max      1920.000000
dtype: float64

In [8]:
activ2019.groupby(by=['finess']).size()[activ2019.groupby(by=['finess']).size()==1]

finess
010780294    1
030002158    1
040784860    1
060010949    1
060021276    1
            ..
970404158    1
970405064    1
970405676    1
970409991    1
980500763    1
Length: 200, dtype: int64

In [9]:
activ2019.groupby(by=['finess']).size()[activ2019.groupby(by=['finess']).size()==1920]

finess
750712184    1920
dtype: int64

Certains établissements sont donc très diversifiés et d'autres très spécialisés. 

On souhaite fusionner activ2019 avec ID (pour les colonnes fi et stjr). Cela nous permettra d'obtenir le statut juridique regroupé de chaque établissement.

In [10]:
activ_fi_stjr= pd.merge(activ2019,ID[["fi",'stjr']],left_on="finess", right_on='fi',how='inner')
activ_fi_stjr

,finess,ghm2,_freq,fi,stjr
0,010007300,02C05J,813,010007300,2
1,010007300,02C08J,87,010007300,2
2,010007300,06K02Z,169,010007300,2
3,010007300,06K04J,663,010007300,2
4,010007300,06K05J,63,010007300,2
...,...,...,...,...,...
56908,970466751,02C11J,230,970466751,2
56909,970466751,03K02J,29,970466751,2
56910,970466751,09C03J,89,970466751,2
56911,970466751,09C09J,23,970466751,2


Cette fusion nous renvoit un Dataframe comprenant 56 913 lignes. Nous avons donc perdu 95 830 lignes suite à cette fusion. En effet, certains établissements de la base activ2019 sont représentés dans la base ID de la SAE par le  numéro finess de l'entité juridique(fi_ej) et non par le finess (fi). 
Chaque établissement de santé est représenté par un numéro finess unique. Cependant plusieurs établissements peuvent être rattachés à une même entité juridique. De ce fait, les numéros fi_ej de la base ID ne sont pas uniques :

In [11]:
print("fi_ej : ",ID["fi_ej"].size,'\n',"fi_ej uniques :",ID["fi_ej"].nunique())

fi_ej :  4018 
 fi_ej uniques : 2231


Vérifions qu'il n'est pas possible d'avoir plusieurs statuts juridiques pour la même entité : 

In [12]:
for i in ID['fi_ej'].unique() : 
    if len(ID['stjr'][ID['fi_ej'] == i]) > 1:
        if int(len(ID['stjr'][ID['fi_ej'] == i])) != int((ID['stjr'][ID['fi_ej'] == i].sum())/(ID['stjr'][ID['fi_ej'] == i].min())) : 
            print("Il peut y avoir différents statuts juridiques par entité")

In [13]:
# On retire les lignes identiques en vue de la fusion 
df = pd.DataFrame({"fi_ej": ID['fi_ej'], 'stjr':ID['stjr'] })
df.drop_duplicates(keep = 'first', inplace=True)

Nous pouvons maintenant fusionner activ2019 en fonction du numéro finess des entités juridiques. 

In [14]:
activ_fiej_stjr=pd.merge(activ2019, df , left_on = ['finess'], right_on = ['fi_ej'],how='inner')
activ_fiej_stjr

,finess,ghm2,_freq,fi_ej,stjr
0,010007987,20Z041,244,010007987,1
1,010007987,20Z042,81,010007987,1
2,010007987,23K02Z,42,010007987,1
3,010007987,23Z02Z,164,010007987,1
4,010008407,01C15J,198,010008407,1
...,...,...,...,...,...
95672,980500003,23M20Z,77,980500003,1
95673,980500003,28Z07Z,1453,980500003,1
95674,980500003,28Z14Z,427,980500003,1
95675,980500003,28Z15Z,698,980500003,1


In [15]:
activ_stjr = pd.concat([activ_fiej_stjr, activ_fi_stjr] )
activ_stjr

,finess,ghm2,_freq,fi_ej,stjr,fi
0,010007987,20Z041,244,010007987,1,NaN
1,010007987,20Z042,81,010007987,1,NaN
2,010007987,23K02Z,42,010007987,1,NaN
3,010007987,23Z02Z,164,010007987,1,NaN
4,010008407,01C15J,198,010008407,1,NaN
...,...,...,...,...,...,...
56908,970466751,02C11J,230,NaN,2,970466751
56909,970466751,03K02J,29,NaN,2,970466751
56910,970466751,09C03J,89,NaN,2,970466751
56911,970466751,09C09J,23,NaN,2,970466751


In [16]:
activ2019['finess'].nunique()

1416

Au total, en combinant activ_fiej_stjr et activ_fi_stjr nous avons 152590  lignes. Il nous manque encore 153 lignes par rapport à la base initiale. Identifions ces éléments :

In [17]:
l=[]
for i in (activ2019['finess'].unique()):
    if activ_fiej_stjr['finess'].str.contains(i).nunique()==1 :
        l.append(i)
m=[]
for i in l :
    if activ_fi_stjr['finess'].str.contains(i).nunique()==1:
        m.append(i)
        print(i)

200200145
560002511
680021680
690022108
690781836
690782834
780150058
920008539
970500005


Ces 153 lignes correspondent à 9 établissements sanitaires.Ils seront omis de notre étude. 

(1 : Public, 2 : Privé lucratif, 3 : Privé non lucratif)

In [18]:
activ_stjr.head()

,finess,ghm2,_freq,fi_ej,stjr,fi
0,010007987,20Z041,244,010007987,1,NaN
1,010007987,20Z042,81,010007987,1,NaN
2,010007987,23K02Z,42,010007987,1,NaN
3,010007987,23Z02Z,164,010007987,1,NaN
4,010008407,01C15J,198,010008407,1,NaN


In [19]:
public = activ_stjr [(activ_stjr.stjr == 1)|(activ_stjr.stjr == 3)]
public

,finess,ghm2,_freq,fi_ej,stjr,fi
0,010007987,20Z041,244,010007987,1,NaN
1,010007987,20Z042,81,010007987,1,NaN
2,010007987,23K02Z,42,010007987,1,NaN
3,010007987,23Z02Z,164,010007987,1,NaN
4,010008407,01C15J,198,010008407,1,NaN
...,...,...,...,...,...,...
56455,970405676,28Z04Z,1469,NaN,3,970405676
56456,970407151,28Z02Z,24,NaN,3,970407151
56457,970407151,28Z03Z,52,NaN,3,970407151
56458,970407151,28Z04Z,2133,NaN,3,970407151


In [20]:
activ_public=pd.merge(public, ghs_pub[['GHM-NRO', 'GHS-LIB','GHS-PRI']] , left_on = ['ghm2'], right_on = ['GHM-NRO'],how='inner')
activ_public

,finess,ghm2,_freq,fi_ej,stjr,fi,GHM-NRO,GHS-LIB,GHS-PRI
0,010007987,20Z041,244,010007987,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","2351,81"
1,010007987,20Z041,244,010007987,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","4558,72"
2,010008407,20Z041,21,010008407,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","2351,81"
3,010008407,20Z041,21,010008407,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","4558,72"
4,010780054,20Z041,83,010780054,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","2351,81"
...,...,...,...,...,...,...,...,...,...
113260,970408589,18M104,20,970408589,1,NaN,18M104,"Maladies infectieuses sévères, niveau 4","8366,47"
113261,380012658,08C57J,34,NaN,3,380012658,08C57J,Libérations articulaires du membre inférieur à...,"2187,98"
113262,920000684,05C092,22,NaN,3,920000684,05C092,"Autres interventions cardiothoraciques, âge in...","12113,04"
113263,920000684,23C023,21,NaN,3,920000684,23C023,Interventions chirurgicales avec autres motifs...,"9230,73"


In [21]:
113265 -109888 

3377

Cette base ne correspond pas à la base "public" initiale : il y a 3377 lignes supplémentaires. Généralement, à une GHM correspondant un seul GHS. Cependant, certains GHM peuvent être facturés par plusieurs GHS. Les écarts de valorisation des GHM peuvent être très significatifs. C'est pourquoi nous devons traiter ces doublons. 

## Traitement des GHM donnant lieu à plus d'un GHS 

In [22]:
# Convertion du prix des ghs en float et non en string
ghs_pub['GHS-PRI'] = [x.replace(",", ".") for x in ghs_pub['GHS-PRI']]
ghs_pub['GHS-PRI']=ghs_pub['GHS-PRI'].astype(float)

On remarque que dans la base ghs_pub il y a au total 2671 ghs dont 2574 valeurs uniques. Il y a donc 97 ghm qui sont présents en doublons. Certains GHM sont facturés par plus de deux ghs. Voici la liste des ghm facturés par au moins deux ghs: 

In [23]:
duplic= ghs_pub[ghs_pub['GHM-NRO'].duplicated()]
duplic['GHM-NRO'].unique()

array(['01C041', '01C042', '01C043', '01C044', '01C121', '01C122',
       '01C123', '01C124', '02C091', '02C092', '02C093', '02C094',
       '02C09J', '02C021', '02C022', '02C023', '02C024', '02C02J',
       '05C061', '05C062', '05C063', '05C064', '05C021', '05C022',
       '05C023', '05C024', '05C031', '05C032', '05C033', '05C034',
       '05C081', '05C082', '05C083', '05C084', '06C19J', '06C191',
       '08C611', '08C612', '08C613', '08C614', '08C621', '08C622',
       '08C623', '08C624', '08C62J', '09C111', '09C112', '09C113',
       '09C114', '09C041', '09C042', '09C043', '09C044', '09C051',
       '09C052', '09C053', '09C054', '09C05J', '11C031', '11C032',
       '11C033', '11C034', '13C12J', '13C16J', '17K041', '19C021',
       '20Z041', '20Z042', '20Z043', '20Z044', '20Z021', '20Z022',
       '20Z023', '20Z024', '23Z02Z', '27C041', '27C042', '27C043',
       '27C044', '27C051', '27C052', '27C053', '27C054', '28Z04Z',
       '28Z11Z', '28Z18Z', '28Z19Z'], dtype=object)

In [24]:
# Création d'un dateframe avec ces valeurs uniques 
df_unique = pd.DataFrame({'GHM-NRO': duplic['GHM-NRO'].unique()})

In [25]:
# Association des ghm à toutes leurs possibilités de facturation
doublons=pd.merge(df_unique, ghs_pub[['GHM-NRO', 'GHS-LIB','GHS-PRI']] , left_on = ['GHM-NRO'], right_on = ['GHM-NRO'],how='inner')
doublons

,GHM-NRO,GHS-LIB,GHS-PRI
0,01C041,"Craniotomies en dehors de tout traumatisme, âg...",5407.74
1,01C041,"Craniotomies en dehors de tout traumatisme, âg...",11950.27
2,01C042,"Craniotomies en dehors de tout traumatisme, âg...",9017.60
3,01C042,"Craniotomies en dehors de tout traumatisme, âg...",15560.14
4,01C043,"Craniotomies en dehors de tout traumatisme, âg...",13020.04
...,...,...,...
179,28Z11Z,"Techniques spéciales d'irradiation externe, en...",761.56
180,28Z18Z,Radiothérapie conformationnelle avec modulatio...,176.53
181,28Z18Z,Radiothérapie conformationnelle avec modulatio...,373.54
182,28Z19Z,Préparations à une irradiation externe par RCM...,1068.19


In [26]:
# Liste des valeurs présentes dans le Dataframe "doublons" mais pas dans le Dataframe "public"
drop_values= []
for i in df_unique['GHM-NRO']: 
    if (i in public['ghm2'].unique()) == False : 
        drop_values.append(i)

In [27]:
# On retire ces valeurs que nous n'utiliserons pas pour ce projet
for i in drop_values :
    Index= doublons[doublons['GHM-NRO']==i].index
    doublons.drop(Index,inplace=True)

In [28]:
doublons

,GHM-NRO,GHS-LIB,GHS-PRI
0,01C041,"Craniotomies en dehors de tout traumatisme, âg...",5407.74
1,01C041,"Craniotomies en dehors de tout traumatisme, âg...",11950.27
2,01C042,"Craniotomies en dehors de tout traumatisme, âg...",9017.60
3,01C042,"Craniotomies en dehors de tout traumatisme, âg...",15560.14
4,01C043,"Craniotomies en dehors de tout traumatisme, âg...",13020.04
...,...,...,...
179,28Z11Z,"Techniques spéciales d'irradiation externe, en...",761.56
180,28Z18Z,Radiothérapie conformationnelle avec modulatio...,176.53
181,28Z18Z,Radiothérapie conformationnelle avec modulatio...,373.54
182,28Z19Z,Préparations à une irradiation externe par RCM...,1068.19


In [29]:
doublons['GHM-NRO'].unique()

array(['01C041', '01C042', '01C043', '01C044', '01C121', '01C122',
       '01C123', '01C124', '02C091', '02C092', '02C09J', '02C021',
       '02C022', '02C02J', '05C061', '05C062', '05C063', '05C064',
       '05C022', '05C023', '05C024', '05C031', '05C032', '05C033',
       '05C034', '05C081', '05C082', '05C083', '05C084', '06C19J',
       '06C191', '08C611', '08C612', '08C613', '08C614', '08C621',
       '08C622', '08C623', '08C624', '08C62J', '09C111', '09C112',
       '09C113', '09C041', '09C042', '09C043', '09C051', '09C052',
       '09C05J', '11C031', '11C032', '11C033', '11C034', '13C12J',
       '13C16J', '17K041', '19C021', '20Z041', '20Z042', '20Z043',
       '20Z044', '20Z021', '20Z022', '20Z023', '23Z02Z', '27C044',
       '27C053', '27C054', '28Z04Z', '28Z11Z', '28Z18Z', '28Z19Z'],
      dtype=object)

In [47]:
GHM_ecart_important = []
for i in doublons['GHM-NRO'].unique(): 
    if (doublons[doublons['GHM-NRO']==i]['GHS-PRI'].std()) >= 2000 : 
        GHM_ecart_important.append(i)
        print (doublons[doublons['GHM-NRO']==i])

  GHM-NRO                                            GHS-LIB   GHS-PRI
0  01C041  Craniotomies en dehors de tout traumatisme, âg...   5407.74
1  01C041  Craniotomies en dehors de tout traumatisme, âg...  11950.27
  GHM-NRO                                            GHS-LIB   GHS-PRI
2  01C042  Craniotomies en dehors de tout traumatisme, âg...   9017.60
3  01C042  Craniotomies en dehors de tout traumatisme, âg...  15560.14
  GHM-NRO                                            GHS-LIB   GHS-PRI
4  01C043  Craniotomies en dehors de tout traumatisme, âg...  13020.04
5  01C043  Craniotomies en dehors de tout traumatisme, âg...  19562.58
  GHM-NRO                                            GHS-LIB   GHS-PRI
6  01C044  Craniotomies en dehors de tout traumatisme, âg...  17050.13
7  01C044  Craniotomies en dehors de tout traumatisme, âg...  23592.66
  GHM-NRO                                            GHS-LIB   GHS-PRI
8  01C121  Craniotomies pour affections non tumorales, âg...   4810.18
9  01C

In [31]:
somme_freq=[]
for i in GHM_ecart_important:
    somme_freq.append(public[public['ghm2']==i]['_freq'].sum())

In [32]:
print("Nombre de séjours impliqués dans la problématique des doublons :",sum(somme_freq),"\n", "Nombre de séjours au total :", activ_public['_freq'].sum(),"\n","Part en pourcentage :",sum(somme_freq)/activ_public['_freq'].sum()*100 )

Nombre de séjours impliqués dans la problématique des doublons : 44972 
 Nombre de séjours au total : 21201875 
 Part en pourcentage : 0.2121133154496949


44 972 séjours vont être impactés par la problèmatique des doublons dans la tarification sur 21 201 875 séjours au total. Cela représente donc moins de 1% des séjours, ce qui est assez négligeable. Cependant, regardons si certains hopitaux sont largement concerné par cette problématique. 

In [33]:
public

,finess,ghm2,_freq,fi_ej,stjr,fi
0,010007987,20Z041,244,010007987,1,NaN
1,010007987,20Z042,81,010007987,1,NaN
2,010007987,23K02Z,42,010007987,1,NaN
3,010007987,23Z02Z,164,010007987,1,NaN
4,010008407,01C15J,198,010008407,1,NaN
...,...,...,...,...,...,...
56455,970405676,28Z04Z,1469,NaN,3,970405676
56456,970407151,28Z02Z,24,NaN,3,970407151
56457,970407151,28Z03Z,52,NaN,3,970407151
56458,970407151,28Z04Z,2133,NaN,3,970407151


In [34]:
df=pd.DataFrame({'Total_sejours':public.groupby(by=['finess'])['_freq'].sum()})
df

,Total_sejours
finess,
010007987,531
010008407,6166
010009132,41
010780054,65011
010780062,4703
...,...
970407151,2209
970408589,139801
970409991,576


In [35]:
df2=pd.DataFrame({'GHM_ecart_important':GHM_ecart_important})
df3=pd.merge(public, df2 , left_on = ['ghm2'], right_on = ['GHM_ecart_important'],how='inner')
df4=pd.DataFrame({'Total_sejours_complexes':df3.groupby(by=['finess'])['_freq'].sum()})
df4

,Total_sejours_complexes
finess,
010007987,81
010780054,170
010780062,31
020004495,24
030780100,70
...,...
970202164,177
970211207,209
970403606,33


In [36]:
df5=pd.merge(df, df4 ,how='inner',left_on='finess',right_on='finess')
df5

,Total_sejours,Total_sejours_complexes
finess,,
010007987,531,81
010780054,65011,170
010780062,4703,31
020004495,2162,24
030780100,27763,70
...,...,...
970202164,350,177
970211207,111034,209
970403606,14639,33


Au total il y a 196 hopitaux publics ou privés à but non lucratif sont impactés par cette problématique des doublons sur 859,  soit environ 22 % des hopitaux. 

In [62]:
df5["Part_séjours_complexes (en %)"] = df5['Total_sejours_complexes']/df5['Total_sejours']*100

In [63]:
df5[df5["Part_séjours_complexes"]>10]

,Total_sejours,Total_sejours_complexes,Part_séjours_complexes,Part_séjours_complexes (en %)
finess,,,,
010007987,531,81,15.254237,15.254237
210780607,424,128,30.188679,30.188679
300780095,167,23,13.772455,13.772455
390781193,416,110,26.442308,26.442308
440028538,571,204,35.726795,35.726795
440042141,297,62,20.875421,20.875421
510000052,476,52,10.924370,10.924370
550000095,191,24,12.565445,12.565445
570000513,347,106,30.547550,30.547550


14 des 196 hopitaux publics ou privés à but non lucratif ont au moins 10% de leurs séjours classés comme "séjours complexes". Deux hopitaux ont jusqu'à 100% de leurs séjours classés en "séjours complexes". 
Cette problèmatique des doublons est donc non négligeable. 

In [64]:
df5.sort_values('Part_séjours_complexes')

,Total_sejours,Total_sejours_complexes,Part_séjours_complexes,Part_séjours_complexes (en %)
finess,,,,
570005165,154511,27,0.017474,0.017474
780001236,77261,21,0.027181,0.027181
950110080,67289,22,0.032695,0.032695
680020336,124254,43,0.034607,0.034607
190000042,64006,23,0.035934,0.035934
...,...,...,...,...
570000513,347,106,30.547550,30.547550
440028538,571,204,35.726795,35.726795
970202164,350,177,50.571429,50.571429


In [50]:
activ2019[activ2019.finess=='590781811']

,finess,ghm2,_freq
81800,590781811,20Z022,74
81801,590781811,20Z042,32


In [66]:
df5[df5.Part_séjours_complexes >=5].shape

(27, 4)

On retirera donc les établissements dont la part de séjours complexes est supérieure à 5%. Et on fixera tous les tarifs multiples à la moyenne de ces tarifs.  

In [40]:
df=activ_public.groupby(['finess'])['_freq'].sum()

In [41]:
activ_public['Nombre_total_séjours']=activ_public.groupby(['finess'])['_freq'].sum()

In [42]:
activ_public

,finess,ghm2,_freq,fi_ej,stjr,fi,GHM-NRO,GHS-LIB,GHS-PRI,Nombre_total_séjours
0,010007987,20Z041,244,010007987,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","2351,81",NaN
1,010007987,20Z041,244,010007987,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","4558,72",NaN
2,010008407,20Z041,21,010008407,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","2351,81",NaN
3,010008407,20Z041,21,010008407,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","4558,72",NaN
4,010780054,20Z041,83,010780054,1,NaN,20Z041,"Ethylisme avec dépendance, niveau 1","2351,81",NaN
...,...,...,...,...,...,...,...,...,...,...
113260,970408589,18M104,20,970408589,1,NaN,18M104,"Maladies infectieuses sévères, niveau 4","8366,47",NaN
113261,380012658,08C57J,34,NaN,3,380012658,08C57J,Libérations articulaires du membre inférieur à...,"2187,98",NaN
113262,920000684,05C092,22,NaN,3,920000684,05C092,"Autres interventions cardiothoraciques, âge in...","12113,04",NaN
113263,920000684,23C023,21,NaN,3,920000684,23C023,Interventions chirurgicales avec autres motifs...,"9230,73",NaN


In [43]:
for i in GHM_ecart_important :
    

SyntaxError: unexpected EOF while parsing (<ipython-input-43-db681d6ddbd3>, line 2)

Voyons si certains correspondent à la base public. 

In [ ]:
public_duplic=pd.merge(public, duplic[['GHM-NRO', 'GHS-LIB','GHS-PRI']] , left_on = ['ghm2'], right_on = ['GHM-NRO'],how='inner')
public_duplic

In [ ]:
public_duplic[(public_duplic.finess=='010007987')]

In [ ]:
ghs_pub['GHM-NRO'].duplicated().value_counts()

In [ ]:
activ_public[(activ_public.ghm2=='20Z041')]

In [ ]:
113265-109888

In [ ]:
ghs_pub['GHM-NRO'].nunique()

In [ ]:
ghs_pub['GHM-NRO'].count()

In [ ]:
l=[]
j=0
for i in ghs_pub['GHM-NRO'].value_counts():
    if i>1 :
        j+=i
        l.append(i)
        

In [ ]:
j

In [ ]:
len(l)

In [ ]:
ghs_pub.head()